In [1]:
# 경고 메세지가 안나오게..
import warnings
warnings.filterwarnings('ignore')

# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# KFold
from sklearn.model_selection import KFold           # 랜덤하게 섞어서 자를 수도 있고, 순서대로 자를 수도 있음
from sklearn.model_selection import StratifiedKFold # 결과 데이터의 비율이 최대한 균등하게 들어갈 수 있도록!

# 교차검증 함수
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# 학습 데이터와 검증 데이터로 나누는 함수
from sklearn.model_selection import train_test_split

# 데이터 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

# 평가함수
from sklearn.metrics import accuracy_score

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

# 군집
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift

# 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 딥러닝
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

# 다중분류를 위한 핫-윈 인코더
from keras.utils import to_categorical

# 저장된 딥러닝 모델을 복구하는 함수
from keras.models import load_model

# epoch마다 모델을 저장하는 함수
from keras.callbacks import ModelCheckpoint

# 더이상 성능 향상이 이루어지지 않는다면 조기 중단시킬 수 있는 함수
from keras.callbacks import EarlyStopping

# 저장
import pickle

# 시간 모듈
import time

# 그래프 설정
plt.rcParams['font.family']='Malgun Gothic'
# 맥용 plt.rcParams['font.family']='AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

In [2]:
# seed 설정
np.random.seed(3)
tf.random.set_seed(3)

In [3]:
# 데이터 불러오기
df1 = pd.read_csv('../../data/wine.csv', header=None)
df1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [4]:
# 데이터 분류

x = df1.drop(12, axis=1)
y = df1[12]

display(x)
display(y)

,0,1,2,3,4,5,6,7,8,9,10,11
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


0       1
1       1
2       1
3       1
4       1
       ..
6492    0
6493    0
6494    0
6495    0
6496    0
Name: 12, Length: 6497, dtype: int64

In [5]:
# 모델 설정
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [6]:
# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
# 자동 중단 설정
# patience : 지정해준 숫자만큼 동안 성능향상이 이루어지지 않았다면 중단
# 손실함수 말고, 정확도로 하고 싶으면 정확도로 해도 됨 - 둘다 하고 싶으면 둘 다 설정해서 하자
# 정확도 보다는 손실률로 하는 것이 더 좋음

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

In [8]:
# 모델 실행
model.fit(x, y, validation_split=0.2, epochs=2000, batch_size=500, callbacks=[early_stopping_callback])

Epoch 1/2000
11/11 [==============================] - 1s 33ms/step - loss: 1.7230 - accuracy: 0.6724 - val_loss: 0.6574 - val_accuracy: 0.6769
Epoch 2/2000
11/11 [==============================] - 0s 11ms/step - loss: 0.6426 - accuracy: 0.6536 - val_loss: 0.0777 - val_accuracy: 0.9954
Epoch 3/2000
11/11 [==============================] - 0s 5ms/step - loss: 0.4765 - accuracy: 0.7516 - val_loss: 0.2655 - val_accuracy: 0.9246
Epoch 4/2000
11/11 [==============================] - 0s 5ms/step - loss: 0.3877 - accuracy: 0.8473 - val_loss: 0.1922 - val_accuracy: 0.9585
Epoch 5/2000
11/11 [==============================] - 0s 5ms/step - loss: 0.3404 - accuracy: 0.8888 - val_loss: 0.1949 - val_accuracy: 0.9477
Epoch 6/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.3023 - accuracy: 0.9076 - val_loss: 0.2056 - val_accuracy: 0.9400
Epoch 7/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.2857 - accuracy: 0.9107 - val_loss: 0.1860 - val_accuracy: 0.9462
Epoc

11/11 [==============================] - 0s 6ms/step - loss: 0.1519 - accuracy: 0.9422 - val_loss: 0.0884 - val_accuracy: 0.9777
Epoch 59/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.1567 - accuracy: 0.9405 - val_loss: 0.0777 - val_accuracy: 0.9846
Epoch 60/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.1407 - accuracy: 0.9437 - val_loss: 0.0887 - val_accuracy: 0.9792
Epoch 61/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.1544 - accuracy: 0.9390 - val_loss: 0.0720 - val_accuracy: 0.9846
Epoch 62/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.1464 - accuracy: 0.9426 - val_loss: 0.0816 - val_accuracy: 0.9808
Epoch 63/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.1461 - accuracy: 0.9428 - val_loss: 0.0872 - val_accuracy: 0.9808
Epoch 64/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.1420 - accuracy: 0.9472 - val_loss: 0.0755 - val_accuracy: 0.9831
Epoch 65/2000

11/11 [==============================] - 0s 6ms/step - loss: 0.0888 - accuracy: 0.9737 - val_loss: 0.0302 - val_accuracy: 0.9915
Epoch 116/2000
11/11 [==============================] - 0s 5ms/step - loss: 0.1035 - accuracy: 0.9679 - val_loss: 0.0447 - val_accuracy: 0.9854
Epoch 117/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0952 - accuracy: 0.9684 - val_loss: 0.1004 - val_accuracy: 0.9738
Epoch 118/2000
11/11 [==============================] - 0s 13ms/step - loss: 0.1020 - accuracy: 0.9716 - val_loss: 0.0577 - val_accuracy: 0.9831
Epoch 119/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0862 - accuracy: 0.9733 - val_loss: 0.0447 - val_accuracy: 0.9854
Epoch 120/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0885 - accuracy: 0.9712 - val_loss: 0.0402 - val_accuracy: 0.9854
Epoch 121/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0849 - accuracy: 0.9704 - val_loss: 0.0620 - val_accuracy: 0.9808
Epoch 

Epoch 172/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0768 - accuracy: 0.9777 - val_loss: 0.0496 - val_accuracy: 0.9823
Epoch 173/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0751 - accuracy: 0.9755 - val_loss: 0.0311 - val_accuracy: 0.9915
Epoch 174/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0737 - accuracy: 0.9745 - val_loss: 0.0451 - val_accuracy: 0.9823
Epoch 175/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0716 - accuracy: 0.9790 - val_loss: 0.0475 - val_accuracy: 0.9800
Epoch 176/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0631 - accuracy: 0.9788 - val_loss: 0.0522 - val_accuracy: 0.9785
Epoch 177/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0688 - accuracy: 0.9784 - val_loss: 0.0510 - val_accuracy: 0.9808
Epoch 178/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0674 - accuracy: 0.9789 - val_loss: 0.0354 - val_accuracy:

Epoch 229/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0599 - accuracy: 0.9780 - val_loss: 0.0421 - val_accuracy: 0.9877
Epoch 230/2000
11/11 [==============================] - 0s 5ms/step - loss: 0.0618 - accuracy: 0.9818 - val_loss: 0.0527 - val_accuracy: 0.9823
Epoch 231/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0615 - accuracy: 0.9826 - val_loss: 0.0332 - val_accuracy: 0.9900
Epoch 232/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0582 - accuracy: 0.9788 - val_loss: 0.0346 - val_accuracy: 0.9900
Epoch 233/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0658 - accuracy: 0.9805 - val_loss: 0.0868 - val_accuracy: 0.9731
Epoch 234/2000
11/11 [==============================] - 0s 12ms/step - loss: 0.0681 - accuracy: 0.9797 - val_loss: 0.0371 - val_accuracy: 0.9900
Epoch 235/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0643 - accuracy: 0.9795 - val_loss: 0.0441 - val_accuracy

Epoch 286/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0568 - accuracy: 0.9849 - val_loss: 0.0609 - val_accuracy: 0.9823
Epoch 287/2000
11/11 [==============================] - 0s 5ms/step - loss: 0.0634 - accuracy: 0.9828 - val_loss: 0.0242 - val_accuracy: 0.9923
Epoch 288/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0682 - accuracy: 0.9793 - val_loss: 0.0383 - val_accuracy: 0.9885
Epoch 289/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0517 - accuracy: 0.9853 - val_loss: 0.0692 - val_accuracy: 0.9800
Epoch 290/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0639 - accuracy: 0.9805 - val_loss: 0.0232 - val_accuracy: 0.9931
Epoch 291/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0583 - accuracy: 0.9792 - val_loss: 0.0398 - val_accuracy: 0.9869
Epoch 292/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0547 - accuracy: 0.9840 - val_loss: 0.0386 - val_accuracy:

Epoch 343/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0622 - accuracy: 0.9827 - val_loss: 0.0366 - val_accuracy: 0.9892
Epoch 344/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0582 - accuracy: 0.9834 - val_loss: 0.0704 - val_accuracy: 0.9785
Epoch 345/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0478 - accuracy: 0.9878 - val_loss: 0.0406 - val_accuracy: 0.9869
Epoch 346/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0584 - accuracy: 0.9828 - val_loss: 0.0273 - val_accuracy: 0.9923
Epoch 347/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0564 - accuracy: 0.9836 - val_loss: 0.0633 - val_accuracy: 0.9808
Epoch 348/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0613 - accuracy: 0.9814 - val_loss: 0.0524 - val_accuracy: 0.9823
Epoch 349/2000
11/11 [==============================] - 0s 6ms/step - loss: 0.0535 - accuracy: 0.9855 - val_loss: 0.0391 - val_accuracy:

In [9]:
model.evaluate(x,y)[1]

204/204 [==============================] - 0s 2ms/step - loss: 0.0692 - accuracy: 0.9781


0.9781437516212463